
# Overview
This CodeLab demonstrates how to build a fused TFLite LSTM model for MNIST recognition using Keras, and how to convert it to TensorFlow Lite.

The CodeLab is very similar to the Keras LSTM [CodeLab](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/examples/experimental_new_converter/keras_lstm.ipynb). However, we're creating fused LSTM ops rather than the unfused versoin.

Also note: We're not trying to build the model to be a real world application, but only demonstrate how to use TensorFlow Lite. You can a build a much better model using CNN models. For a more canonical lstm codelab, please see [here](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).


# Step 0: Prerequisites
It's recommended to try this feature with the newest TensorFlow nightly pip build.

# Step 1: Build the MNIST LSTM model.

In [13]:
import sys
sys.path.append('/path/to')

import random
import numpy as np
import os
import datetime
import click
from utils import mdrnn_config

import mdrnn as mdrnn
from tensorflow import keras
import tensorflow as tf

In [19]:
    # Model Hyperparameters
    SEQ_LEN = 50
    SEQ_STEP = 1
    TIME_DIST = True
    # Training Hyperparameters:
    VAL_SPLIT = 0.10
    # Set random seed for reproducibility
    SEED = 2345

    # Directly defining "training" function's inputs
    dimension = 9
    modelsize = "s"
    earlystopping = True
    patience = 10
    numepochs = 3
    batchsize = 64


    model_config = mdrnn_config(modelsize)
    mdrnn_units = model_config["units"]
    mdrnn_layers = model_config["layers"]
    mdrnn_mixes = model_config["mixes"]

    print("Model size:", modelsize)
    print("Units:", mdrnn_units)
    print("Layers:", mdrnn_layers)
    print("Mixtures:", mdrnn_mixes)

    random.seed(SEED)
    np.random.seed(SEED)

    # Load dataset
    dataset_location = '../datasets/'
    dataset_filename = f'training-dataset-{str(dimension)}d.npz'

    with np.load(dataset_location + dataset_filename, allow_pickle=True) as loaded:
        perfs = loaded['perfs']

    print("Loaded perfs:", len(perfs))
    print("Num touches:", np.sum([len(l) for l in perfs]))
    corpus = perfs  # might need to do some processing here...processing
    # Restrict corpus to sequences longer than the corpus.
    corpus = [l for l in corpus if len(l) > SEQ_LEN+1]
    print("Corpus Examples:", len(corpus))
    # Prepare training data as X and Y.
    slices = []
    for seq in corpus:
        slices += mdrnn.slice_sequence_examples(seq,
                                                    SEQ_LEN+1,
                                                    step_size=SEQ_STEP)
    X, y = mdrnn.seq_to_overlapping_format(slices)
    X = np.array(X) * mdrnn.SCALE_FACTOR
    y = np.array(y) * mdrnn.SCALE_FACTOR

    print("Number of training examples:")
    print("X:", X.shape)
    print("y:", y.shape)

    # Setup Training Model
    model = mdrnn.build_model(seq_len=SEQ_LEN,
                                hidden_units=mdrnn_units,
                                num_mixtures=mdrnn_mixes,
                                layers=mdrnn_layers,
                                out_dim=dimension,
                                time_dist=TIME_DIST,
                                inference=False,
                                compile_model=True,
                                print_summary=True)

    model_dir = "models/"
    model_name = "musicMDRNN" + "-dim" + str(dimension) + "-layers" + str(mdrnn_layers) + "-units" + str(mdrnn_units) + "-mixtures" + str(mdrnn_mixes) + "-scale" + str(mdrnn.SCALE_FACTOR)
    date_string = datetime.datetime.today().strftime('%Y%m%d-%H_%M_%S')

    filepath = model_dir + model_name + "-E{epoch:02d}-VL{val_loss:.2f}.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')
    terminateOnNaN = keras.callbacks.TerminateOnNaN()
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)
    tboard = keras.callbacks.TensorBoard(log_dir='./logs/' + date_string + model_name,
                                        histogram_freq=0,
                                        write_graph=True,
                                        update_freq='epoch')

    callbacks = [checkpoint, terminateOnNaN, tboard]
    if earlystopping:
        print("Enabling Early Stopping.")
        callbacks.append(early_stopping)
    # Train
    history = model.fit(X, y, batch_size=batchsize,
                        epochs=numepochs,
                        validation_split=VAL_SPLIT,
                        callbacks=callbacks)

Model size: s
Units: 64
Layers: 2
Mixtures: 5
Loaded perfs: 46
Num touches: 94603
Corpus Examples: 33
Number of training examples:
X: (92873, 50, 9)
y: (92873, 50, 9)
Building EMPI Model...
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 50, 9)]           0         
                                                                 
 lstm0 (LSTM)                (None, 50, 64)            18944     
                                                                 
 lstm1 (LSTM)                (None, 50, 64)            33024     
                                                                 
 td_mdn (TimeDistributed)    (None, 50, 95)            6175      
                                                                 
Total params: 58143 (227.12 KB)
Trainable params: 58143 (227.12 KB)
Non-trainable params: 0 (0.00 Byte)
_______________________________

/home/ninjatimtam/.cache/pypoetry/virtualenvs/impsy-0RgZpL3H-py3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1306/1307 [============================>.] - ETA: 0s - loss: 22.3445
Epoch 2: val_loss improved from 19.93788 to 19.07641, saving model to models/musicMDRNN-dim9-layers2-units64-mixtures5-scale10-E02-VL19.08.hdf5
1307/1307 [==============================] - 102s 78ms/step - loss: 22.3445 - val_loss: 19.0764
Epoch 3/3
1307/1307 [==============================] - ETA: 0s - loss: 18.7162
Epoch 3: val_loss improved from 19.07641 to 15.73800, saving model to models/musicMDRNN-dim9-layers2-units64-mixtures5-scale10-E03-VL15.74.hdf5
1307/1307 [==============================] - 106s 81ms/step - loss: 18.7162 - val_loss: 15.7380


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28), name='input'),
    tf.keras.layers.LSTM(20, time_major=False, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# Step 2: Train & Evaluate the model.
We will train the model using MNIST data.

In [ ]:
# Load MNIST dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# Change this to True if you want to test the flow rapidly.
# Train with a small dataset and only 1 epoch. The model will work poorly
# but this provides a fast way to test if the conversion works end to end.
_FAST_TRAINING = False
_EPOCHS = 5
if _FAST_TRAINING:
  _EPOCHS = 1
  _TRAINING_DATA_COUNT = 1000
  x_train = x_train[:_TRAINING_DATA_COUNT]
  y_train = y_train[:_TRAINING_DATA_COUNT]

model.fit(x_train, y_train, epochs=_EPOCHS)
model.evaluate(x_test, y_test, verbose=0)

# Step 3: Convert the Keras model to TensorFlow Lite model.

In [24]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = batchsize
STEPS = SEQ_LEN
INPUT_SIZE = dimension 
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: keras_lstm/assets


INFO:tensorflow:Assets written to: keras_lstm/assets
2024-05-20 18:34:34.672434: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-20 18:34:34.672501: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-05-20 18:34:34.672714: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: keras_lstm
2024-05-20 18:34:34.685635: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-20 18:34:34.685701: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: keras_lstm
2024-05-20 18:34:34.726090: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-05-20 18:34:34.865839: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: keras_lstm
2024-05-20 18:34:34.960719: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }

# Step 4: Check the converted TensorFlow Lite model.
Now load the TensorFlow Lite model and use the TensorFlow Lite python interpreter to verify the results.

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(TEST_CASES):
  expected = model.predict(x_test[i:i+1])
  interpreter.set_tensor(input_details[0]["index"], x_test[i:i+1, :, :])
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result, decimal=5)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()

# Step 5: Let's inspect the converted TFLite model.

Let's check the model, you can see the LSTM will be in it's fused format.

![Fused LSTM](https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/examples/experimental_new_converter/keras_lstm.png)
